In [27]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

In [121]:
data_df = pd.read_csv('./Steam.csv', sep=',', names=["User ID","Name","Type","Hours"], engine='python')
#data_df = data_df[data_df["Type"] == "purchase"]


unique_GameID = data_df['Name'].unique()
unique_UserID = data_df['User ID'].unique()

j = 0
user_old2new_id_dict = dict()
for u in unique_UserID:
    user_old2new_id_dict[u] = j
    j += 1

j = 0
game_old2new_id_dict = dict()
for i in unique_GameID:
    game_old2new_id_dict[i] = j
    j += 1

# Then, use the generated dictionaries to reindex UserID and MovieID in the data_df
for j in range(len(data_df)):
    data_df.at[j, 'User ID'] = user_old2new_id_dict[data_df.at[j, 'User ID']]
   
    
    data_df.at[j, 'Name'] = game_old2new_id_dict[data_df.at[j, 'Name']]

data_df = data_df[data_df["Type"] == "purchase"]

In [122]:
test_df = data_df.sample(frac = 0.3, random_state = np.random.randint(1024))
train_df = data_df.drop(test_df.index)

print ("The testing dataframe is of size: " + str(len(test_df)))
print ("The training dataframe is of size: " + str(len(train_df)))

The testing dataframe is of size: 38853
The training dataframe is of size: 90658


In [123]:
num_user = len(data_df['User ID'].unique())
num_game = len(data_df['Name'].unique())

data_mat = coo_matrix((data_df['Hours'].values, (data_df['User ID'].values, data_df['Name'].values)), shape=(num_user, num_game)).toarray().astype(float)
train_mat = coo_matrix((train_df['Hours'].values, (train_df['User ID'].values, train_df['Name'].values)), shape=(num_user, num_game)).toarray().astype(float)
test_mat = coo_matrix((test_df['Hours'].values, (test_df['User ID'].values, test_df['Name'].values)), shape=(num_user, num_game)).toarray().astype(float)

In [124]:
popularity = np.sum(data_mat,axis=0)

inv_names = {v: k for k, v in game_old2new_id_dict.items()}


top = []
#print (list(movie_old2new_id_dict))
for i in range (len(popularity)):
    #print (i)
    if (i in inv_names):
        top.append((popularity[i],inv_names[i]))

top.sort()
top.reverse()
top2 = []
for i in top[:61]:
    top2.append(i[1])


In [132]:
import csv

train_mat_plus = np.copy(train_mat)

with open("GGs 489 survey (Responses) - Form Responses 1.csv", newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

data = data[1:]

for i in range (len(data)):
    data[i][1] = data[i][1].split(", ")
print (data[0][1])

for i in range (len(data)):
    k = np.zeros((1,num_game))
    
    for j in data[i][1]:
        k[0,game_old2new_id_dict[j]] = 1

    train_mat_plus = np.vstack([train_mat_plus, k])


['Dota 2', 'Counter-Strike', 'Counter-Strike Condition Zero', 'War Thunder', 'Terraria', 'PlanetSide 2', 'Team Fortress Classic', 'Call of Duty Modern Warfare 2', 'Metro 2033', 'Left 4 Dead', 'Grand Theft Auto V']


In [133]:
print (train_mat.size/num_game)
print (train_mat_plus.size/num_game)

12393.0
12394.0
